# Introduction for Deep learning practicioners new to remote sensing

- explain that there are largely two types of datasets in torchgeo
- explain vision datasets, explain geodatasets (most of them do not contain raw satellite imagery, need to pair)
- highlight difference between them
- what is special about remote sensing imagery as compared to imagenet
- crs units explanation
- projection that transforms the surface of a 3D earth onto a 2D planar image that we are used to
- as mentioned, remote sensing datasets consist of large tiles, how to do tiling (process of splitting a larger tile into smaller patches or chips)
- explain how this works in torchgeo
- why the chips can be of non-constant size
- how to return constant size chips that can be part of a dataloader
- so how to construct a dataloader with geosampler
- how to create datasplits if they are not predefined or not available
- how to use torchgeo models with and without pytorch lightning modules

The aim of this tutorial is to make users with a Deep Learning background that are new to remote sensing familiar with the TorchGeo library. In general, remote sensing datasets often contain much higher dimensional images than datasets the reader might be used to. Additionall, Deep Learning models often require a given image input dimension and therefore, a main objective of this tutorial will be to demonstrate how to do this for all types of TorchGeo Datasets.

It is recommended to run this notebook on Google Colab if you do not have your own GPU. Click the "Open in Colab" butto above to get started.

## Setup

Install TorchGeo.

In [1]:
#%pip install torchgeo

## Imports

Import libraries that we require for this tutorial.

In [2]:
import os
import tempfile

from torch.utils.data import DataLoader

## TorchGeo Datasets

In TorchGeo, we differentiate between two types of datasets: Geospatial Datasets and Non-geospatial Datasets. While the former includes geospatial information that we will look at in more detail, the latter do not. List other differences... We will now take a closer look at Non-geospatial datasets before we turn our attention to Geospatial datasets.



## Non-geospatil Datasets

For people familiar with torchvision datasets and their combination with Pytorch DataLoaders, TorchGeo's Non-geospatial datasets will work the same way. For example, to use the .... dataset you can simply do the following:

In [3]:
# show a non-geospatial dataset loaded

By design, TorchGeo returns a dictionary that contains a sample of the chosen dataset. Furthermore, we can also pass our custom transformations when instantiating a dataset. If we would now like to obtain a custom image size, we can use a custom transform. So let's do that.

In [4]:
# custom image dataset size transform

Now, let's apply this transform.

In [5]:
# use the transform and demonstrate

As, we can see, samples from the dataloader now have the desired dimensions. Hopefully, you now have a good understanding of how TorchGeo's Non-geospatial datasets work, how easy they are to use, and how you can adapt a dataset to your liking through custom transforms. We will now turn our attention to Geospatial datasets.

## Geospatial Datasets

Geospatial Datasets, implemented as GeoDataset class in TorchGeo, contain geospatial information such as latitude, longitude, coordinate stystem, and projection. Furthermore, Geospatial Datasets can contain Raster or Vector Data (Explain difference here). When working with remote sensory data such as satellite imagery, there occurs a projection that transforms the surface of the 3D Globe onto a 2D image that we are used to seeing. Additionally, the data should include the precise geographical location of a captured image. For this purpose cartographers have developed a framework called the coordinate reference system (CRS), which aims to display the spherical earh in just two dimensions. Due to this, all projections will yield some distortion of distance, area, or angular properties! A nice example of, how different CRS distort a given map, can be found here. There exist many different CRS (link here) that each have their advantages and disadvantages depending on the location, scale, and purpose of a map. As a consequence, TorchGeo's Geospatial datasets have a crs parameter, that lets you project the given dataset as you deem useful. 

Another substantial difference between TorchGeo's Geospatial Datasets and common Pytorch datasets is that the GeoDataset does not index the dataset with an integer, but rather with a Bounding Box consisting of *geographical* coordinates. More specifically, every GeoDataset has a dataset.index object which is an R-tree (link here) containing the Bounding Boxes and filenames of all raster or vector files in the dataset. In TorchGeo there exists a custom GeoSampler that takes care of the indexing into geospatial datsets.

Before we finally take a closer look at how to set up the data pipeline, it should be mentioned that a majority of TorchGeo's geospatial datasets do not contain both images and labels, as you might expect, but only one of the top. This means that if we are interested in a certain TorchGeo dataset that only contains labels and want to train a model to make predictions from raw satellite imagery, we need to pair this particular dataset with an available imagery dataset of the same resolution. In many cases, TorchGeo can do this automatically. 

KInda follow the getting started tutorial, but with more elaborate explanations and a focus on constant sample sizes.